In [21]:
# Cuts used to go from o.summary.fits to o.gst.fits

#snr = 5.0
#sharp = 0.04
#crowd = 0.5
#objtype = 1
#flag = 99

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
import numpy as np


In [4]:
path='test_photometry/'

In [113]:
photometry=fits.open(path+'o.gst.fits')[1]
photometry_sum=fits.open(path+'o.summary.fits')[1]

In [6]:
tbl=Table(photometry.data)
tbl.colnames

['chip',
 'RA',
 'DEC',
 'X',
 'Y',
 'OBJECT_TYPE',
 'F555W_VEGA',
 'F555W_ERR',
 'F555W_SNR',
 'F555W_SHARP',
 'F555W_ROUND',
 'F555W_CROWD',
 'F555W_FLAG']

In [114]:
tbl_2=Table(photometry_sum.data)
tbl_2.colnames

['chip',
 'RA',
 'DEC',
 'X',
 'Y',
 'OBJECT_TYPE',
 'F555W_VEGA',
 'F555W_ERR',
 'F555W_SNR',
 'F555W_SHARP',
 'F555W_ROUND',
 'F555W_CROWD',
 'F555W_FLAG']

In [29]:
sn2021sjt_position = SkyCoord(ra=309.33000691667*u.degree, dec=+66.106417016667*u.degree)

In [52]:
sn2021sjt_position

<SkyCoord (ICRS): (ra, dec) in deg
    (309.33000692, 66.10641702)>

In [72]:
catalog = SkyCoord(ra=tbl['RA']*u.degree, dec=tbl['DEC']*u.degree)
test_object = SkyCoord(ra= 309.30106986096405*u.degree, dec=+66.12665483424358*u.degree)

In [32]:
# Now idx are indices into catalog that are the closest objects to each of the coordinates in c
# d2d are the on-sky distances between them,
# d3d are the 3-dimensional distances.
# Because coordinate objects support indexing, idx enables easy access to the matched set of coordinates 
# in the catalog:

dra, ddec = sn2021sjt_position.spherical_offsets_to(matches)

In [221]:
max_sep = 1 * u.arcsec
#idx, d2d, d3d = sn2021sjt_position.match_to_catalog_sky(catalog)
d2d = sn2021sjt_position.separation(catalog)
catalogmsk = d2d < max_sep
idxcatalog = np.where(catalogmsk)[0]
print(idxcatalog)
print(len(idxcatalog))
objlist = np.zeros((len(idxcatalog),2))
for i in range(len(idxcatalog)):
    objlist[i][0]=tbl['RA'][idxcatalog[i]]
    objlist[i][1]=tbl['DEC'][idxcatalog[i]]
print(objlist)

[3309 6656 6746 6753 7057]
5
[[309.32997059  66.10642008]
 [309.33034943  66.10635611]
 [309.33047245  66.10643374]
 [309.33067157  66.10635759]
 [309.32996709  66.10656332]]


In [105]:
d3d

<Quantity 2.62284035e-07>

In [197]:
tbl['RA'][7671]

309.3323900457097

In [198]:
tbl['DEC'][7671]

66.101105240825

In [215]:
def writeregionfile(filename, objlist, color="blue",sys=''):
    if sys == '': sys = 'wcs'
    out = open(filename,'w')
    i = -1
    out.write('# Region file format: DS9 version 4.0\nglobal color='+color+' font="helvetica 10 normal" select=1 highlite=1 edit=1 move=1 delete=1 include=1 fixed=0 source\n')
    if sys == 'wcs':
      out.write('fk5\n')
      for ob in objlist:
        i += 1
        out.write("point(%.7f,%.7f) # point=boxcircle text={%i}\n" % (objlist[i][0], objlist[i][1], i))
    if sys == 'img':
      out.write('image\n')
      for ob in objlist:
        i += 1
        out.write("point(%.3f,%.3f) # point=boxcircle text={%i}\n" % (ob.x, ob.y, i))
    out.close()

In [222]:
writeregionfile("nearstars2021sjt1arc.reg", objlist, "red", "wcs")